In [ ]:
# Basic module
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm # progress bar

# PyTorch
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [ ]:
# print version of PyTorch
torch.__version__, torchvision.__version__

#### Prepare CIFAR10 Dataset

*   torch vision datasets: https://pytorch.org/vision/stable/datasets.html
*   CIFAR10 label

0: airplane
1: automobile
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck

In [ ]:
# Define Parameters
NUM_CLASS = 10
# Class name and class mapping
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]
class_map = {cls: i for i, cls in enumerate(class_names)}
print(class_map)

#### torch.utils.data.Dataset

https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset

*   讀取**1**筆資料
*   輸出torch.Tensor (張量)
* Datasets provided by torchvision https://pytorch.org/vision/stable/datasets.html


In [ ]:
# Download dataset
train_ds = torchvision.datasets.CIFAR10('data', # saved path
    train=True, # training or testing set
    download=True # download dataset from internet
)
val_ds = torchvision.datasets.CIFAR10('data',
    train=False,
    download=True
)

In [ ]:
# 資料筆數
print('Number of training   samples:', len(train_ds))
print('Number of validation samples:', len(val_ds))

In [ ]:
# 隨機取出1筆資料
idx = np.random.randint(low=0, high=len(train_ds))
img, label = train_ds[idx]

print(idx)
print(type(img), type(label))

In [ ]:
# Convert to np.ndarray and show image
img_np = np.array(img)
print('img shape: ', img_np.shape)
print('label: ', label)
print('class name: ', class_names[label])
plt.imshow(img_np)
plt.show()

#### Data Proprocess

**transforms.ToTensor()**

1.   PIL.Image to torch.FloatTensor (張量)
  
    *   Input: PIL Image or numpy.ndarray (H, W, C) in the range [0, 255]
    *   Output: torch.FloatTensor (C, H, W) in the range [0.0, 1.0]

2.   TODO: 資料擴增, ... etc

**NOTE**: PyTorch 要求通道數在前的形式 (C, H, W)


In [ ]:
preprocess = transforms.Compose([
    transforms.ToTensor(), # Convert PIL.Image or np.array to torch.Tensor
])

In [ ]:
# Build dataset with data preprocess
train_ds = torchvision.datasets.CIFAR10('data',
    train=True,
    download=True,
    transform=preprocess # 前處理
)
val_ds = torchvision.datasets.CIFAR10('data',
    train=False,
    download=True,
    transform=preprocess # 前處理
)

#### Dataset + DataLoader

**torch.utils.data.DataLoader**: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

* 組成批次(**batch**)
* 資料取樣
* 讀取順序 (shuffle)

In [ ]:
# 使用 DataLoader 讀取批次資料
BATCH_SIZE = 1024
train_dataloader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True, # 訓練每輪結束打亂順序
)
val_dataloader = torch.utils.data.DataLoader(
    val_ds,
    batch_size=BATCH_SIZE
)

檢查資料shape

N: 批次數量 (batch)

C: 通道數

H: 高度

W: 寬度

**PyTorch use channel first !**

In [ ]:
for x, y in train_dataloader:
    print("type ", type(x), type(y))
    print("Shape of x (N, C, H, W): ", x.shape, x.dtype)
    print("Shape of y (N, ): ", y.shape, y.dtype)
    break

In [ ]:
# 取出批次的第0筆顯示
plt.imshow(x[0].permute(1, 2, 0)) # (C H W) -> (H W C)
plt.title(str(y[0]))
plt.show()

#### Build Model

In [ ]:
# 取得現有硬體 ('GPU', 'CPU')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

IMG_SIZE = 32

定義模型

In [ ]:
# 序列化模型: 一層一層直接連接
model = nn.Sequential(
    nn.Flatten(), # (3, 32, 32) -> (3*32*32, )
    nn.Linear(
        in_features=IMG_SIZE*IMG_SIZE*3,
        out_features=64), # (C*H*W) -> (64)
    nn.ReLU(),
    nn.Linear(64, 128), # (64) -> (128)
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, NUM_CLASS), # (128) -> NUM_CLASS
)
model = model.to(device)

In [ ]:
 # 繼承 nn.Module
class NeuralNet(nn.Module):
    def __init__(self):
        # 網路層初始化
        super().__init__()
        self.flatten = nn.Flatten() # (C, H, W) -> (C*H*W)
        self.base_model = nn.Sequential(
            nn.Linear(in_features=IMG_SIZE*IMG_SIZE*3, out_features=64), # (C*H*W) -> (64)
            nn.ReLU(),
            nn.Linear(64, 128), # (64) -> (128)
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, NUM_CLASS), # (128) -> (NUM_CLASS)
        )
    def forward(self, x):
        # forward函式定義輸入的資料張量該用如何運算
        # 輸入資料x: (bs, 3, 32, 32)
        # 輸入模型都是torch.Tensor
        # 且都帶有批次數量的維度 batch_size

        x = self.flatten(x)
        # flatten: (bs, 3, 32, 32) -> (bs, 3072)
        logits = self.base_model(x)
        # base_model: (bs, 3072) -> (bs, 10)

        return logits

In [ ]:
# 初始化模型，移到device
model = NeuralNet().to(device)

In [ ]:
print(model)

#### 訓練(學習): 最佳化模型參數

In [ ]:
# 損失函數: 計算誤差
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# 優化器: 更新模型參數
optimizer = torch.optim.SGD(
    params=model.parameters(), # 要最佳化的模型參數
    lr=1e-1, # learning rate(學習率): 1e-1, 1e-2, 1e-3 ...
)

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # 資料總數
    num_batches = len(dataloader) # 切成批次數

    model.train() # 模型轉成訓練模式
    epoch_loss, epoch_correct = 0, 0

    # 依序取出每批資料
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # 資料搬到device

        # 資料丟到模型預測
        pred = model(x)
        # 計算損失
        loss = loss_fn(pred, y)

        optimizer.zero_grad() # 將過去累積梯度歸零
        loss.backward() # 透過loss反向傳播計算梯度
        optimizer.step() # 更新模型參數

        # 寫紀錄
        epoch_loss += loss.item() # tensor -> python value
        # pred: (N, Class)
        # 計算類別最大值位置(index)是否與解答相同, 統計總數
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # 計算平均loss, Accuracy
    return epoch_loss/num_batches, epoch_correct/size

def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # 模型轉成測試模式
    epoch_loss, epoch_correct = 0, 0

    # 測試時不需要計算梯度
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
EPOCHS = 100 # 訓練總回合數, 每一回合都看完所有資料一遍
logs = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}
for epoch in tqdm(range(EPOCHS)):
    train_loss, train_acc = train_epoch(train_dataloader, model, loss_fn, optimizer)
    val_loss, val_acc = test_epoch(val_dataloader, model, loss_fn)

    print(f'EPOCH: {epoch} \
    train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
    val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)

#### Logs

In [ ]:
# Plot loss curve
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(logs['train_loss'])
plt.plot(logs['val_loss'])
plt.legend(['train_loss', 'val_loss'])
# plot acc
plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(logs['train_acc'])
plt.plot(logs['val_acc'])
plt.legend(['train_acc', 'val_acc'])
plt.show()

#### Save Model

Saving & Loading Model (weights only)

**Recommended**

In [ ]:
# 取得模型參數
model.state_dict()

In [ ]:
PATH = './model_weights.pth' # .pt
# 存參數
torch.save(model.state_dict(), PATH)

# 讀參數
model.load_state_dict(torch.load(PATH))

Saving & Loading Model (entire model)

In [ ]:
MODEL_PATH = './model.pth'
# 存模型
torch.save(model, MODEL_PATH)
# 讀模型
model = torch.load(MODEL_PATH)

#### Evaluation

In [ ]:
# load model
model = NeuralNet()
model.load_state_dict(torch.load(PATH)) # 讀取參數
model.eval()

In [ ]:
# take first 10 images
n = 10
for (images, labels) in val_dataloader:
    images, labels = images[:n], labels[:n]
    images_grid = torchvision.utils.make_grid(images[:n])
    images_grid = images_grid.permute(1, 2, 0) # (C, H, W) -> (H, W, C)
    plt.imshow(images_grid.numpy())
    break

In [ ]:
# take first 2 images
n = 2
for (images, labels) in val_dataloader:
    images, labels = images[:n], labels[:n]
    break

# Predict by model
with torch.no_grad():
    pred = model(images) # predict logits
print('raw_prediction logits', pred, pred.shape, sep="\n")

In [ ]:
pred_softmax = nn.Softmax(dim=1)(pred) # 模型輸出轉乘機率值
print('prediction after softmax', pred_softmax, pred_softmax.shape, sep="\n")

In [ ]:
# max_prob: 每一筆資料最大機率值
# predicted_cls: 最大值idx
max_prob, predicted_cls = torch.max(pred_softmax, dim=1)

In [ ]:
predicted_cls = pred_softmax.argmax(dim=1)

In [ ]:
predicted_cls

In [ ]:
print('GroundTruth: ', ' '.join(class_names[labels[j]] for j in range(n)))
print('Prediction: ', ' '.join(class_names[predicted_cls[j]] for j in range(n)))

In [ ]:
# 使用 torchsummary 顯示模型架構
import torchsummary

torchsummary.summary(
    model.to(device), # 模型
    input_size=(3, 32, 32) # 一筆輸入資料形狀
)

In [ ]:
# 使用 torchinfo 顯示模型架構
!pip install torchinfo

import torchinfo
torchinfo.summary(
    model,
    input_size=(BATCH_SIZE, 3, 32, 32)
)